![Hand with calculator](calculator.jpg)

Did you know that the average return from investing in stocks is [10% per year](https://www.nerdwallet.com/article/investing/average-stock-market-return) (not accounting for inflation)? But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

## dates
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `date_joined` | The date that the company became a unicorn.  |
| `year_founded` | The year that the company was founded.       |

## funding
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| `company_id`       | A unique ID for the company.                 |
| `valuation`        | Company value in US dollars.                 |
| `funding`          | The amount of funding raised in US dollars.  |
| `select_investors` | A list of key investors in the company.      |

## industries
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `company_id`   | A unique ID for the company.                 |
| `industry`     | The industry that the company operates in.   |

## companies
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| `company_id`   | A unique ID for the company.                      |
| `company`      | The name of the company.                          |
| `city`         | The city where the company is headquartered.      |
| `country`      | The country where the company is headquartered.   |
| `continent`    | The continent where the company is headquartered. |


In [5]:
-- The task will be broken down using two CTEs 
-- The best industry is found by counting the number of 'unicorns' in each industry and selecting the 3 highest ranking industries 

WITH best_industry AS (
 SELECT i.industry, 
		COUNT(*) AS num_unicorns -- counts the number of 'unicorns'
FROM industries AS i
LEFT JOIN dates AS d -- joins the dates table to the industries table
USING(company_id)
WHERE EXTRACT(year FROM d.date_joined) between 2019 AND 2021 -- filters for years 2019, 2020 and 2021
GROUP BY i.industry
ORDER BY num_unicorns DESC
LIMIT 3),

-- The average valuation for these industries are now calculated 

second AS (SELECT i.industry, 
		EXTRACT(year FROM d.date_joined) AS year,
		COUNT(*) AS num_unicorns,
		ROUND(AVG(f.valuation/1000000000),2) AS average_valuation_billions -- calculates the average valuations in billions and rounds it to two decimals
FROM industries AS i
LEFT JOIN dates AS d -- joins the dates table to the industries table
USING(company_id)
LEFT JOIN funding AS f -- joins the funding table to the industries table
USING(company_id)
WHERE EXTRACT(year FROM d.date_joined) between 2019 AND 2021
GROUP BY i.industry, year
ORDER BY year DESC)

-- Now the CTEs will be joined to produce the final result. 

SELECT 	s.industry,
		s.year,
		s.num_unicorns,
		s.average_valuation_billions
FROM best_industry AS b
JOIN second AS s
ON b.industry = s.industry; -- joins both CTEs

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,Fintech,2021,138,2.75
2,Internet software & services,2021,119,2.15
3,E-commerce & direct-to-consumer,2020,16,4.00
4,Fintech,2020,15,4.33
5,Internet software & services,2020,20,4.35
6,E-commerce & direct-to-consumer,2019,12,2.58
7,Fintech,2019,20,6.80
8,Internet software & services,2019,13,4.23
